This file just tests the rotate and measure function in Trajectree. Due to the integral nature of this function in all measurements, it is important that this works correctly. 

In [1]:
from trajectree.fock_optics.utils import create_vacuum_state
from trajectree.fock_optics.devices import ry, rx, rz, global_phase
from trajectree.fock_optics.measurement import rotate_and_measure
from trajectree.fock_optics.outputs import print_quantum_state, read_quantum_state
from trajectree.fock_optics.noise_models import single_mode_bosonic_noise_channels, depolarizing_operators, two_qubit_depolarizing_channel

from trajectree.trajectory import trajectory_evaluator, quantum_channel

import numpy as np
from scipy import sparse as sp
import qutip as qt
from numpy import sqrt
from quimb.tensor.tensor_arbgeom import tensor_network_apply_op_vec #type: ignore
from quimb.tensor import MatrixProductOperator as mpo #type: ignore

import matplotlib.pyplot as plt
import matplotlib.animation as animation

import copy
from IPython.display import HTML

/home/asingal/home/.venv/lib/python3.10/site-packages/numba/np/ufunc/parallel.py:371: NumbaWarning: The TBB threading layer requires TBB version 2021 update 6 or later i.e., TBB_INTERFACE_VERSION >= 12060. Found TBB_INTERFACE_VERSION = 12050. The TBB threading layer is disabled.
  warnings.warn(problem)
/home/asingal/home/.venv/lib/python3.10/site-packages/cotengra/hyperoptimizers/hyper.py:33: UserWarning: Couldn't import `kahypar` - skipping from default hyper optimizer and using basic `labels` method instead.
  warnings.warn(


In [2]:
def create_perfect_bell_state(N=3, error_tolerance = 1e-10):
    # Defining the control and target inputs here
    a_dag = qt.create(N).full()
    I = qt.identity(N).full()

    a_H = np.kron(a_dag, I)
    a_V = np.kron(I, a_dag)

    # Testing the case with only one MPS for both control and target
    vacuum = create_vacuum_state(num_modes=4, N=N)
    pure_EPPS_operator = 1/sqrt(2) * (np.kron(a_H, a_H) + np.kron(a_V, a_V))
    pure_EPPS_operator_MPO = mpo.from_dense(pure_EPPS_operator, dims = N, sites = (0,1,2,3), L=4, tags="ideal_state")    
    psi = tensor_network_apply_op_vec(pure_EPPS_operator_MPO, vacuum, compress=True, contract = True, cutoff = error_tolerance)
    return psi, psi.site_tags



In [15]:
N = 2
idler = [np.pi/4, 0]
signal = [np.pi/2]
num_modes = 4
efficiency = 1
error_tolerance = 1e-10
num_simulations = 1

print(np.round(ry(signal[0], N, return_unitary = False, site1 = 0, site2 = 1, total_sites=2).to_dense(), 2))
print(np.round(ry(idler[0], N, return_unitary = False, site1 = 0, site2 = 1, total_sites=2).to_dense(), 2))

[[ 1.  +0.j  0.  +0.j  0.  +0.j  0.  +0.j]
 [-0.  +0.j  0.71+0.j -0.71+0.j  0.  +0.j]
 [ 0.  +0.j  0.71+0.j  0.71+0.j  0.  +0.j]
 [ 0.  +0.j  0.  +0.j  0.  +0.j  1.  +0.j]]
[[ 1.  +0.j  0.  +0.j  0.  +0.j  0.  +0.j]
 [ 0.  +0.j  0.92+0.j -0.38+0.j  0.  +0.j]
 [ 0.  +0.j  0.38+0.j  0.92+0.j  0.  +0.j]
 [ 0.  +0.j  0.  +0.j  0.  +0.j  1.  +0.j]]


In [16]:
psi, site_tags = create_perfect_bell_state(N)

read_quantum_state(psi, N, num_states = 2)

Corresponding Basis terms:
1H0V_B 1H0V_A - 5 - [0.70710678+0.j]
0H1V_B 0H1V_A - 10 - [0.70710678+0.j]


In [18]:
quantum_channel_list = []

phase_flip_ops = [sp.csr_matrix(global_phase(0, N, return_unitary = True, site1 = 0, site2 = 1, total_sites = 2))]
quantum_channel_list.append(quantum_channel(N = N, num_modes = 8, formalism = "kraus", kraus_ops_tuple = ((0,1), phase_flip_ops), name = "first_kraus"))

coincidence_HH_ops_list = rotate_and_measure(None, N, site_tags, num_modes, efficiency, error_tolerance, idler, signal, rotations = {"signal":(0,1), "idler":(2,3)}, measurements = {1:(0,3), 0:(1,2)}, return_MPOs = True)
quantum_channel_list.extend([quantum_channel(N = N, num_modes = 8, formalism = "closed", unitary_MPOs = PA_MPO) for PA_MPO in coincidence_HH_ops_list])

evaluator = trajectory_evaluator(quantum_channel_list)

psi_iter = evaluator.perform_simulation(psi, error_tolerance, normalize = False)

read_quantum_state(psi_iter, N, num_states = 2)

psi_iter.normalize()

coincidence_HH_ops_list

Corresponding Basis terms:
0H1V_B 1H0V_A - 9 - [0.27059805-0.j]


[MatrixProductOperator(tensors=2, indices=5, L=4, max_bond=4),
 MatrixProductOperator(tensors=2, indices=5, L=4, max_bond=4),
 MatrixProductOperator(tensors=1, indices=2, L=4, max_bond=None),
 MatrixProductOperator(tensors=1, indices=2, L=4, max_bond=None),
 MatrixProductOperator(tensors=1, indices=2, L=4, max_bond=None),
 MatrixProductOperator(tensors=1, indices=2, L=4, max_bond=None)]